Example of dataloader usage. Unsure of how to load skin segmentation data at this time.

In [11]:
# Datasest dir should look like:


# ├── data
# │   ├── v1_cam1_no_split
# |      ├── Adjust Radio
# |          ├── 1.jpg
# │      ├── Drink
# |           ├── 1.jpg
# |      ├── Test_data_list.csv
# |      ├── Train_data_list.csv
# |
# │   ├── v2_cam1_cam2_ split_by_driver
# |      ├── Camera 1
# |          ├── test
# |          ├── train
# │      ├── Camera 2
# |          ├── test
# |          ├── train
# |      ├── skin_nonskin_pixels


In [12]:
from dataloaders import *
import torchvision.transforms as T


transform = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224)),
])

v1_train = V1Dataset(csv_path='data/v1_cam1_no_split/Train_data_list.csv', images_folder='data/v1_cam1_no_split', 
                     transform=transform)
v1_test = V1Dataset(csv_path='data/v1_cam1_no_split/Test_data_list.csv', images_folder='data/v1_cam1_no_split', 
                    transform=transform)

v2_train = V2Dataset(cam1_path='data/v2_cam1_cam2_ split_by_driver/Camera 1/train', 
                     cam2_path='data/v2_cam1_cam2_ split_by_driver/Camera 2/train', transform=transform)
v2_test = V2Dataset(cam1_path='data/v2_cam1_cam2_ split_by_driver/Camera 1/test', 
                    cam2_path='data/v2_cam1_cam2_ split_by_driver/Camera 2/test', transform=transform)

v1_trainloader = DataLoader(v1_train, batch_size=32, shuffle=True)
v1_testloader = DataLoader(v1_test, batch_size=32, shuffle=True)

v2_trainloader = DataLoader(v2_train, batch_size=32, shuffle=True)
v2_testloader = DataLoader(v2_test, batch_size=32, shuffle=True)

In [13]:
print(f'Len of v1_trainloader: {len(v1_trainloader)}')
for images, label in v1_trainloader:
    print(images.shape)
    print(label.shape)
    break


Len of v1_trainloader: 406


/home/ron/miniconda3/envs/cybertruck-env/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([32, 3, 224, 224])
torch.Size([32])


In [14]:
print(f'Len of v1_trainloader: {len(v1_testloader)}')
for images, label in v1_testloader:
    print(images.shape)
    print(label.shape)
    break

Len of v1_trainloader: 136
torch.Size([32, 3, 224, 224])
torch.Size([32])


In [15]:
print(f'Len of v2_trainloader: {len(v2_trainloader)}')
for images, label in v2_trainloader:
    print(images.shape)
    print(label.shape)
    break

Len of v2_trainloader: 393


torch.Size([32, 3, 224, 224])
torch.Size([32])


In [16]:
print(f'Len of v1_trainloader: {len(v2_testloader)}')
for images, label in v2_testloader:
    print(images.shape)
    print(label.shape)
    break

Len of v1_trainloader: 61
torch.Size([32, 3, 224, 224])
torch.Size([32])
